In [1]:
import sys
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import collections
import argparse
from Bio import SeqIO
from itertools import product

In [4]:

def headerNAC(k, foutput):
    global listTotal, caracteres
    listTotal = []
    file = open(foutput, 'a')
    file.write('%s,' % ('nameseq'))
    permsList = [''.join(str(i) for i in x) for x in product(caracteres, repeat=k)]
    # print (permsList)
    for perm in permsList:
        # print (perm)
        listTotal.append(perm)
        file.write('%s,' % (str(perm)))
    file.write('label')
    file.write('\n')
    return


def chunks(seq, win, step):
    seqlen = len(seq)
    for i in range(0, seqlen, step):
        j = seqlen if i+win>seqlen else i+win
        yield seq[i:j]
        if j==seqlen: break
    return        
    

def chunks_two(seq, win):
    seqlen = len(seq)
    for i in range(seqlen):
        j = seqlen if i+win>seqlen else i+win
        yield seq[i:j]
        if j==seqlen: break
    return

            
def file_record(name_seq, foutput, labelDataset):
    file = open(foutput, 'a')
    file.write('%s,' % (name_seq))
    for x in probabilities:
        # print (x)
        file.write('%s,' % (str(x[1])))
    file.write(labelDataset)
    file.write('\n')
    print ('Recorded Sequence: %s' % (name_seq))
    return
    

# Nucleic acid composition (NAC), Di-nucleotide composition (DNC), Tri-nucleotide composition (TNC)
def nacSeq(k, foutput, finput, labelDataset):
    global probabilities
    headerNAC(k, foutput)
    for seq_record in SeqIO.parse(finput, 'fasta'):
        seq = seq_record.seq
        seq = seq.upper()	
        name_seq = seq_record.name
        probabilities = []
        kmer = {}
        totalWindows = (len(seq) - k) + 1 # (L - k + 1)
        permsList = [''.join(str(i) for i in x) for x in product(caracteres, repeat=k)]
        for key in permsList:
            kmer[key] = 0
        kmer = collections.OrderedDict(sorted(kmer.items()))
        for subseq in chunks_two(seq, k):
            if subseq in kmer:
                # print(subseq)
                kmer[subseq] = kmer[subseq] + 1
            else:
                kmer[subseq] = 1
        for key, value in kmer.items():
            # print (key)
            # print (value)
            probabilities.append([str(key), value/totalWindows])
        file_record(name_seq, foutput, labelDataset)
    return


def ExtractionTechniques(finput, foutput, tech):
    global listTotal, caracteres, complement
    labelDataset = 'DNA'
    # tech = str(args.type)
    seq = 1
    stepw = 1
    if seq == 1:
        caracteres = ['A', 'C', 'G', 'T']
        complement = {'A': 'T','C': 'G','G': 'C','T': 'A'}
    else:
        caracteres = ['A', 'C', 'G', 'U']
        complement = {'A': 'U','C': 'G','G': 'C','U': 'A'}
    if tech == 'NAC' or tech == 'nac':
        nacSeq(1, foutput, finput, labelDataset)
    elif tech == 'DNC' or tech == 'dnc':
        nacSeq(2, foutput, finput, labelDataset)
    elif tech == 'TNC' or tech == 'tnc':
        nacSeq(3, foutput, finput, labelDataset)
#############################################################################

In [5]:
ExtractionTechniques("res.fasta", 'res_1.csv', 'dnc')

NameError: name 'caracteres' is not defined